# 微分可能な MatterSim - バッチ処理のデモ

このノートブックでは、MatterSim のバッチ処理機能を使って、複数の結晶を同時に処理し、入力（atom_types, lattice, positions）に対して勾配を計算する方法を示します。

## 概要

- **目的**: 複数の結晶を同時に処理し、バッチ最適化を行う
- **新しい API**: `DifferentiableMatterSimCalculator.forward_batch()`
- **主な機能**: 
  - 可変原子数のバッチ処理
  - atom_types, lattice, positions への勾配計算
  - Soft normalization による完全な勾配伝播

In [ ]:
# Python 環境のセットアップと診断
import sys
import subprocess
from pathlib import Path

# 現在の環境情報
print("="*70)
print("Python 環境情報")
print("="*70)
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print(f"Current directory: {Path.cwd()}")

# プロジェクトルートを特定
project_root = Path.cwd().parent
src_path = project_root / "src"
print(f"Project root: {project_root}")
print(f"Source path: {src_path}")

# mattersim のインストール状態を確認
print("\n" + "="*70)
print("パッケージインストール状態")
print("="*70)

try:
    import mattersim
    print(f"✓ mattersim is installed")
    print(f"  Location: {mattersim.__file__}")
except ImportError:
    print("✗ mattersim is NOT installed")

# threebody_indices の確認
try:
    from mattersim.datasets.utils import threebody_indices
    print(f"✓ threebody_indices module is available")
except ImportError as e:
    print(f"✗ threebody_indices module is NOT available: {e}")
    print("\n" + "="*70)
    print("解決策: 以下のいずれかを実行してください")
    print("="*70)
    print("1. Jupyter カーネルを再起動してください（Kernel → Restart Kernel）")
    print("2. または、以下のセルのコメントを外して実行してください：")
    print("")
    print("# 次のセルで実行:")
    print(f"# !cd {project_root} && pip install Cython && pip install -e .")
    raise

print("\n" + "="*70)
print("セットアップ完了")
print("="*70)

In [ ]:
# 必要なライブラリのインポート
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from ase.build import bulk

from mattersim.forcefield.differentiable_potential import (
    DifferentiableMatterSimCalculator,
    sizes_to_batch_index,
)

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

## 1. ヘルパー関数の確認

まず、バッチ処理に必要な `sizes_to_batch_index` 関数を確認します。

In [ ]:
# sizes から batch_index への変換
sizes = torch.tensor([2, 3, 1])  # 3つの結晶: 2原子, 3原子, 1原子
batch_index = sizes_to_batch_index(sizes)

print(f"sizes: {sizes}")
print(f"batch_index: {batch_index}")
print(f"\n各原子がどの結晶に属するか: {batch_index.tolist()}")

## 2. バッチ処理の基本

Diamond Si と FCC-Fe の2つの結晶をバッチ処理します。

In [ ]:
# 2つの結晶を準備
si = bulk("Si", "diamond", a=5.43)
fe = bulk("Fe", "fcc", a=3.6)
atoms_list = [si, fe]

print(f"Si: {len(si)} atoms")
print(f"Fe: {len(fe)} atoms")
print(f"Total: {len(si) + len(fe)} atoms")

In [ ]:
# Calculator を初期化
device = "cpu"  # または "cuda"
calc = DifferentiableMatterSimCalculator(device=device)
print("Calculator が初期化されました")

In [ ]:
# バッチ入力を準備
sizes = torch.tensor([len(si), len(fe)])

# atom_types: concatenate
atom_types_si = F.one_hot(torch.tensor(si.get_atomic_numbers()), 95).float()
atom_types_fe = F.one_hot(torch.tensor(fe.get_atomic_numbers()), 95).float()
atom_types = torch.cat([atom_types_si, atom_types_fe], dim=0)

# positions: concatenate
positions = torch.cat([
    torch.tensor(si.get_positions(), dtype=torch.float32),
    torch.tensor(fe.get_positions(), dtype=torch.float32)
], dim=0)

# lattice: stack
lattice = torch.stack([
    torch.tensor(si.cell.array, dtype=torch.float32),
    torch.tensor(fe.cell.array, dtype=torch.float32)
], dim=0)

print(f"atom_types shape: {atom_types.shape}  # (sum(N_i), 95)")
print(f"positions shape:  {positions.shape}   # (sum(N_i), 3)")
print(f"lattice shape:    {lattice.shape}     # (nb_graphs, 3, 3)")
print(f"sizes:            {sizes}             # [N_1, N_2]")

## 3. バッチ forward の実行

`forward_batch` メソッドで2つの結晶のエネルギーを同時に計算します。

In [ ]:
# バッチ forward（勾配なし）
output = calc.forward_batch(
    atoms_list,
    atom_types=atom_types,
    positions=positions,
    lattice=lattice,
    sizes=sizes,
    include_forces=False,
    soft_normalize=False,  # まずは標準モード
)

energies = output["total_energy"]
print(f"\nバッチエネルギー:")
print(f"  Si: {energies[0].item():.6f} eV")
print(f"  Fe: {energies[1].item():.6f} eV")

## 4. 勾配の計算

atom_types, lattice, positions の3つすべてに対して勾配を計算します。

In [ ]:
# requires_grad を設定
atom_types_grad = atom_types.clone().requires_grad_(True)
positions_grad = positions.clone().requires_grad_(True)
lattice_grad = lattice.clone().requires_grad_(True)

# Forward with soft_normalize=True（atom_types への勾配を維持）
output = calc.forward_batch(
    atoms_list,
    atom_types=atom_types_grad,
    positions=positions_grad,
    lattice=lattice_grad,
    sizes=sizes,
    include_forces=False,
    soft_normalize=True,
)

energies = output["total_energy"]
loss = energies.sum()

# Backward
loss.backward()

# 勾配確認
print(f"\n勾配計算結果:")
print(f"  atom_types.grad is not None: {atom_types_grad.grad is not None}")
print(f"  positions.grad is not None:  {positions_grad.grad is not None}")
print(f"  lattice.grad is not None:    {lattice_grad.grad is not None}")

print(f"\n勾配ノルム:")
print(f"  atom_types: {torch.norm(atom_types_grad.grad).item():.6e}")
print(f"  positions:  {torch.norm(positions_grad.grad).item():.6e}")
print(f"  lattice:    {torch.norm(lattice_grad.grad).item():.6e}")

print("\n✓ 3変数すべてに勾配が計算されました！")

## 5. バッチ最適化のデモ

2つの結晶の positions を同時に最適化します。

In [ ]:
# 初期位置に摂動を加える
torch.manual_seed(42)
positions_opt = positions.clone()
positions_opt = positions_opt + torch.randn_like(positions_opt) * 0.02
positions_opt.requires_grad_(True)

# Optimizer
optimizer = torch.optim.Adam([positions_opt], lr=0.02)

# 最適化の履歴
energy_history = []

print("最適化を開始...")
num_steps = 30
for step in range(num_steps):
    optimizer.zero_grad()
    
    output = calc.forward_batch(
        atoms_list,
        positions=positions_opt,
        sizes=sizes,
        include_forces=False,
    )
    energies = output["total_energy"]
    loss = energies.sum()
    
    energy_history.append(energies.detach().cpu().numpy())
    
    loss.backward()
    optimizer.step()
    
    if step % 5 == 0:
        print(f"Step {step:2d}: Si = {energies[0].item():.6f} eV, Fe = {energies[1].item():.6f} eV")

print(f"\n最適化完了！")
print(f"Si エネルギー変化: {energy_history[0][0]:.6f} → {energy_history[-1][0]:.6f} eV")
print(f"Fe エネルギー変化: {energy_history[0][1]:.6f} → {energy_history[-1][1]:.6f} eV")

## 6. エネルギーの推移を可視化

In [ ]:
energy_history = np.array(energy_history)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Si のエネルギー推移
ax1.plot(energy_history[:, 0], 'b-', linewidth=2, label='Si')
ax1.set_xlabel('最適化ステップ', fontsize=12)
ax1.set_ylabel('エネルギー (eV)', fontsize=12)
ax1.set_title('Diamond Si のエネルギー推移', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Fe のエネルギー推移
ax2.plot(energy_history[:, 1], 'r-', linewidth=2, label='Fe')
ax2.set_xlabel('最適化ステップ', fontsize=12)
ax2.set_ylabel('エネルギー (eV)', fontsize=12)
ax2.set_title('FCC-Fe のエネルギー推移', fontsize=14)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ 2つの結晶が同時に最適化されました！")

## 7. 連続分布 atom_types のデモ

atom_types を one-hot ではなく、連続的な確率分布として扱います。

In [ ]:
# logits をパラメータ化
torch.manual_seed(123)
logits_si = torch.zeros(len(si), 95)
logits_si[:, 14] = 5.0  # Si (原子番号14)
logits_fe = torch.zeros(len(fe), 95)
logits_fe[:, 26] = 5.0  # Fe (原子番号26)

logits = torch.cat([logits_si, logits_fe], dim=0)
logits += torch.randn_like(logits) * 0.5  # ノイズを加える
logits.requires_grad_(True)

print("初期 logits:")
print(f"  Si[0, 13:16]: {logits[0, 13:16].detach()}")
print(f"  Fe[2, 25:28]: {logits[2, 25:28].detach()}")

# Optimizer
optimizer_logits = torch.optim.SGD([logits], lr=0.1)

# 最適化
for step in range(10):
    optimizer_logits.zero_grad()
    
    atom_types_soft = F.softmax(logits, dim=1)
    
    output = calc.forward_batch(
        atoms_list,
        atom_types=atom_types_soft,
        sizes=sizes,
        include_forces=False,
        soft_normalize=True,
    )
    
    energies = output["total_energy"]
    
    # 損失: エネルギー + regularization
    atomic_numbers = torch.cat([
        torch.tensor(si.get_atomic_numbers()),
        torch.tensor(fe.get_atomic_numbers())
    ], dim=0)
    target = F.one_hot(atomic_numbers, num_classes=95).float()
    loss = energies.sum() + 0.1 * F.mse_loss(atom_types_soft, target)
    
    loss.backward()
    optimizer_logits.step()

print("\n最終 logits:")
print(f"  Si[0, 13:16]: {logits[0, 13:16].detach()}")
print(f"  Fe[2, 25:28]: {logits[2, 25:28].detach()}")

atom_types_final = F.softmax(logits, dim=1)
print("\n最終 atom_types (確率分布):")
print(f"  Si[0, 13:16]: {atom_types_final[0, 13:16].detach()}")
print(f"  Fe[2, 25:28]: {atom_types_final[2, 25:28].detach()}")

print("\n✓ 連続分布の atom_types で勾配更新が動作しました！")

## 8. まとめ

このノートブックでは、以下を実演しました：

1. ✅ **ヘルパー関数**: `sizes_to_batch_index` による batch_index 生成
2. ✅ **バッチ forward**: 複数結晶の同時エネルギー計算
3. ✅ **3変数の勾配**: atom_types, lattice, positions すべてに勾配計算
4. ✅ **バッチ最適化**: 2つの結晶を同時に最適化
5. ✅ **可視化**: エネルギー推移のプロット
6. ✅ **連続分布 atom_types**: 非one-hot での勾配更新

### 主なポイント

- `forward_batch()` を使用してバッチ処理
- `soft_normalize=True` で atom_types への完全な勾配伝播
- `sizes` または `batch_index` で可変原子数に対応
- PyTorch の optimizer で入力を更新

### 次のステップ

- より複雑な損失関数を定義（例: 目標構造との距離）
- 3つ以上の結晶のバッチ処理
- lattice の制約付き最適化（正定値性の維持など）

詳細なドキュメントは `docs/DIFFERENTIABLE_API.md` を参照してください。